In [1]:
!pip install datasets
!pip install transformers
# !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.8 MB/s eta 0:00

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'deeplearning/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

models_folder="trained_models"

Mounted at /content/drive
['shortjokes.csv', 'openseminar_GAN.pptm', 'openseminar_GAN.pptm.gslides', 'Untitled0.ipynb', 'openseminar_GAN_final.docx', 'Davinci003.ipynb', 'Untitled', 'GPTNEO.ipynb', 'baseline.ipynb', 'RM_Training.ipynb', 'trained_models', 'Transformers.ipynb', 'QA_RL_PPO.ipynb', 'api.ipynb', 'RLHF_Dataset (1).ipynb', 'GPTNeoMyself.ipynb', 'InstructMyself2.ipynb', 'q-learning.ipynb', 'deepdai', 'VAE.ipynb', 'diffusion', 'competition', 'GPTNeo_RLHF.ipynb', 'CRS', 'reward_logs.txt']


In [3]:
import pandas as pd
import re
import io
import json
import os
import ast

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

import transformers
from transformers import AdamW, get_scheduler, GPTNeoModel, GPT2LMHeadModel
import datetime
from datasets import load_dataset

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [4]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M", bos_token='<|startoftext|>', eos_token='<|endoftext|>')
tokenizer.pad_token = tokenizer.eos_token

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# get fine-tuned model?
model_path = GOOGLE_DRIVE_PATH + models_folder + f"/gpt_last_ft.pt"
model.load_state_dict(torch.load(model_path)) # 여기서는 dict만 불러온다. 애초에 dict만 저장함.

model.to(device)
model.eval()
print("skip")

skip


In [6]:
BATCH_SIZE=4
EPOCHS = 5
# LEARNING_RATE = 0.00001
LEARNING_RATE = 0.00001
WARMUP_STEPS = 0
MAX_SEQ_LEN = 1500

In [7]:
import json

class EvidenceLoader(torch.utils.data.Dataset):
  def __init__(self):
    super(EvidenceLoader, self).__init__()
    self.data_list = []
    self.eos_token = "<|endoftext|>"
    # Open the JSON file for reading
    with open(GOOGLE_DRIVE_PATH + '/deepdaiharry_sum.json', 'r') as f:
      load = f.read()
      train_data = load.strip('][').split('", "')

    # Now you can access the data in the dictionary
    for jd in train_data:
      if "\"][\"" in jd:
        self.data_list.append("Assistant:"+jd.split("\"][\"")[0])
        self.data_list.append("Assistant:"+jd.split("\"][\"")[1])
      else:
        self.data_list.append("Assistant:"+jd.strip('"'))
  
  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, idx):
    text=self.data_list[idx]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=100, padding="max_length") #pad => special token X

    return inputs, text

In [8]:
edd = EvidenceLoader()
ed_loader = torch.utils.data.DataLoader(edd, batch_size=BATCH_SIZE, shuffle=True)

FileNotFoundError: ignored

In [ ]:
# Load all helpfulness/harmless subsets (share the same schema)
train_dataset = load_dataset("Anthropic/hh-rlhf", split="train")
test_dataset= load_dataset("Anthropic/hh-rlhf", split="test")

class QaDataset(torch.utils.data.Dataset):
  def __init__(self):
    super().__init__()

    self.data_list = []
    self.end_of_text_token = "<|endoftext|>"
    
    x = 0
    for td in train_dataset:
      text = td['chosen'].strip('\n\n')
      conv = text.split("Human:")
      history=""
      for conversation in conv[1:]:
        conversation = history+"Human:"+conversation
        history = conversation[:]
        if len(conversation)<450 and len(conversation)>10:
          # only one conversation
          data_str = f"{conversation}{self.end_of_text_token}"
          self.data_list.append(data_str)

  def __len__(self):
      return len(self.data_list)

  def __getitem__(self, index):
      text = self.data_list[index]
      inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=400, padding="max_length") #pad => special token X

      return inputs, text # text is not used during training, it's just for check

In [ ]:
# Anthropic, QA dataset
dataset = QaDataset()
qa_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
class InstructLoader(torch.utils.data.Dataset):
  def __init__(self):
    super(InstructLoader, self).__init__()
    self.data_list = []
    self.eos_token = "<|endoftext|>"

    loaded = load_dataset("yahma/alpaca-cleaned")
    train_data = loaded['train']

    temp=[]
    for d in train_data:
      if d['input'] and len(d['input'])>1:
        prompt=f"Humam:{d['instruction']}\n\nInformation:{d['input']}\n\nAssistant:{d['output']}"
      else:
        prompt=f"Humam:{d['instruction']}\n\nAssistant:{d['output']}"
      self.data_list.append(prompt)
  
  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, idx):
    text=self.data_list[idx]
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=350, padding="max_length") #pad => special token X

    return inputs, text

In [10]:
idd = InstructLoader()
id_loader = torch.utils.data.DataLoader(idd, batch_size=4, shuffle=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/yahma___json/yahma--alpaca-cleaned-30c4987849371bac/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
def test(prompt):
  tokenized = tokenizer(prompt, return_tensors="pt")

  input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)

  generated = model.generate(
      input_ids, max_length=1024, num_beams=3, no_repeat_ngram_size=2, early_stopping=True, temperature=0.9,
  )

  return tokenizer.decode(generated[0], skip_special_tokens=True)

In [12]:
print(test(idd[0][1].split("Assistant:")[0]+"Assistant:"))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Humam:Give three tips for staying healthy.

Assistant:1. Make sure to get enough rest and exercise regularly. 
2. Get enough sleep and rest, even if it's just for a few minutes.


In [13]:
from transformers import AdamW

# num of training steps = epoch * lengh of data
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE,  betas=(0.9, 0.95))
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

num_training_steps = 2*len(id_loader)

# scheduler
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model.train()

print("skip")

for epoch in range(0,5):
  batch_count = 0
  print(f"EPOCH {epoch} started" + '=' * 30)

  for idx, (data, text) in tqdm(enumerate(id_loader)):
    # this data is returned from tokenizer => three types.
    input_ids, attention_mask = data['input_ids'].to(device), data['attention_mask'].to(device)

    #forward process
    outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

    # what is loss and logits?
    loss, logits = outputs.loss, outputs.logits
    loss.backward()

    sum_loss = sum_loss + loss.detach().data

    # update optimizer for weight, scheduler for learning_rate
    optimizer.step()
    scheduler.step()

    # reset grads to 0 before another training
    optimizer.zero_grad()
    model.zero_grad()

    batch_count += 1

    if batch_count == 300:
      # model.eval()
      print(f"sum loss {sum_loss}")
      batch_count = 0
      sum_loss = 0.0
      

      model.eval()
      test_prompts = [{
          "prompt":"Who are you?",
          "info":"Your name is Harry Potter, and You are a half-blood wizard. I was born into a world where a prophecy named me as the only one who could defeat the most feared Dark wizard, Lord Voldemort."
        },{
          "prompt":"Tell me about your friends.",
          "info":"At Hogwarts, You made lifelong friends with Ron Weasley and Hermione Granger, and You became the youngest Seeker in a century on the Gryffindor Quidditch team."
        },{
            "prompt":"Tell me about your children.",
            "info":"I married Ginny and we have three children: James Sirius, Albus Severus, and Lily Luna."
        }]
      for pp in test_prompts:
        prompt = "Human:"+pp['prompt']+"\n\nInformation:"+pp['info']+"\n\nAssistant:"
        tokenized = tokenizer(prompt, return_tensors="pt")

        input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)

        generated = model.generate(
            input_ids, max_length=1024, num_beams=1, no_repeat_ngram_size=3, early_stopping=True, temperature=0.9,
        )

        response = tokenizer.decode(generated[0], skip_special_tokens=True)
        print(response)
      model.train()

      # Store the model after each epoch to compare the performance of them
      torch.save(model.state_dict(), GOOGLE_DRIVE_PATH + models_folder + f"/gpt_alasta_{epoch}.pt")

skip
EPOCH 0 started==============================


299it [01:57,  2.49it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sum loss 172.3347625732422


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Human:Who are you?

Information:Your name is Harry Potter, and You are a half-blood wizard. I was born into a world where a prophecy named me as the only one who could defeat the most feared Dark wizard, Lord Voldemort.

Assistant:You are ahalf-blood, wizard, and you are the only wizard in the world.


...

9899it [1:07:59,  2.50it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


sum loss 171.1405487060547


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Human:Who are you?

Information:Your name is Harry Potter, and You are a half-blood wizard. I was born into a world where a prophecy named me as the only one who could defeat the most feared Dark wizard, Lord Voldemort.

Assistant:You are ahalf-blood, wizard, and you are a part of the wizarding world.
Human:Tell me about your friends.

Information:At Hogwarts, You made lifelong friends with Ron Weasley and Hermione Granger, and You became the youngest Seeker in a century on the Gryffindor Quidditch team.

Assistant:Your friends include Ron Weasley, Hermione Granger and Ron Weasley.
Human:Tell me about your children.

Information:I married Ginny and we have three children: James Sirius, Albus Severus, and Lily Luna.

Assistant:My children are my children.


10040it [1:08:58,  2.49it/s]

In [ ]:
torch.save(model.state_dict(), GOOGLE_DRIVE_PATH + models_folder + f"/gpt_harry_talk_{epoch}.pt")

In [ ]:
def inference(prompt, info):
  prompt = "Human:"+prompt+"\n\nInformation:"+info+"\n\nAssistant:"
  tokenized = tokenizer(prompt, return_tensors="pt")

  input_ids, attention_mask = tokenized['input_ids'].to(device), tokenized['attention_mask'].to(device)

  generated = model.generate(
      input_ids, max_length=1024, num_beams=5, no_repeat_ngram_size=3, early_stopping=True, temperature=0.3,
  )

  return tokenizer.decode(generated[0], skip_special_tokens=True)

In [ ]:
def test_harry():
  model.eval()
  test_prompts = [{
      "prompt":"",
      "info":"My name is Harry Potter, and You are a half-blood wizard. I was born into a world where a prophecy named me as the only one who could defeat the most feared Dark wizard, Lord Voldemort."
    },{
      "prompt":"",
       "info":"At Hogwarts, I made lifelong friends with Ron Weasley and Hermione Granger, and I became the youngest Seeker in a century on the Gryffindor Quidditch team."
    },{
        "prompt":"Which sports do you like the most? Use the given",
        "info":"I married Ginny and we have three children: James Sirius, Albus Severus, and Lily Luna."
    }]
  for pp in test_prompts:
    response = inference(pp['prompt'], pp['info'])
    print(response)